In [34]:
path_xlsx_parte = r'C:\Documents\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\PARTE DIARIO TRICHOGRAMMA 2024_RES.xlsx'

In [1]:
import sys
sys.path.append('../_amigocloud')

In [2]:
from amigocloud import AmigoCloud
import pandas as pd
from shapely import wkb

import geopandas as gpd
from shapely.geometry import Point, Polygon, MultiPolygon, MultiPoint
#from shapely.geometry import Point, MultiPoint
import os
#from fastkml import kml

In [3]:
# ruta de carpera donde estan las llaves
ruta = open('../ruta.txt')
ruta = ruta.readlines()[0]

In [4]:
# leer token
acceso = open(ruta + '/_keys/api_amigocloud.key','r')
api_token = acceso.readlines()[0]
acceso.close()
amigocloud = AmigoCloud(token=api_token)
amigocloud

In [102]:
query = {'query': 'SELECT\
    g.id, p.idd, p.fecha_registro, p.piloto_1, p.piloto_2, p.hora_inicio, p.hora_fin, p.labor, p.temperatura, p.viento, p.humedad, p.estado_de_operacion, p.dron, p.obs FROM dataset_342996 p JOIN dataset_345601 g ON ST_Intersects(p.ubicaciones, g.geometry) WHERE p.fecha_registro BETWEEN \'2024-07-04 14:50:55\' AND \'2024-07-04 14:50:55\''}
select = amigocloud.get('https://app.amigocloud.com/api/v1/projects/33457/sql', query)
data = select['data']

In [104]:
df_lib = pd.DataFrame(data)

In [105]:
df_lib

,id,idd,fecha_registro,piloto_1,piloto_2,hora_inicio,hora_fin,labor,temperatura,viento,humedad,estado_de_operacion,dron,obs
0,1,30,2024-07-04 14:50:55+00:00,EDUARDO REYES,CHRISTIAN CALIZAYA,2024-07-04 00:50:55+00:00,2024-07-04 09:00:55+00:00,LIBERACION,20,2,80,CONCLUIDO,DA 01,None
1,2,30,2024-07-04 14:50:55+00:00,EDUARDO REYES,CHRISTIAN CALIZAYA,2024-07-04 00:50:55+00:00,2024-07-04 09:00:55+00:00,LIBERACION,20,2,80,CONCLUIDO,DA 01,None
2,3,30,2024-07-04 14:50:55+00:00,EDUARDO REYES,CHRISTIAN CALIZAYA,2024-07-04 00:50:55+00:00,2024-07-04 09:00:55+00:00,LIBERACION,20,2,80,CONCLUIDO,DA 01,None
3,4,30,2024-07-04 14:50:55+00:00,EDUARDO REYES,CHRISTIAN CALIZAYA,2024-07-04 00:50:55+00:00,2024-07-04 09:00:55+00:00,LIBERACION,20,2,80,CONCLUIDO,DA 01,None
4,5,30,2024-07-04 14:50:55+00:00,EDUARDO REYES,CHRISTIAN CALIZAYA,2024-07-04 00:50:55+00:00,2024-07-04 09:00:55+00:00,LIBERACION,20,2,80,CONCLUIDO,DA 01,None
5,6,30,2024-07-04 14:50:55+00:00,EDUARDO REYES,CHRISTIAN CALIZAYA,2024-07-04 00:50:55+00:00,2024-07-04 09:00:55+00:00,LIBERACION,20,2,80,CONCLUIDO,DA 01,None


In [118]:
lista_idd = list(set(df_lib['idd']))
lista_idd

[30]

In [124]:
lista_id = list(set(df_lib['id']))
lista_id

[1, 2, 3, 4, 5, 6]

In [106]:
nulos = df_lib.isnull().sum()
nulos

id                     0
idd                    0
fecha_registro         0
piloto_1               0
piloto_2               0
hora_inicio            0
hora_fin               0
labor                  0
temperatura            0
viento                 0
humedad                0
estado_de_operacion    0
dron                   0
obs                    6
dtype: int64

In [107]:
df_except_obs = df_lib.drop(columns=['obs'])
null_check = df_except_obs.isnull().any().any()

In [108]:
if null_check:
    df_lib = None
else:
    None

In [109]:
df_parte = pd.read_excel(path_xlsx_parte)

In [110]:
df_parte

,N° LABOR,FECHA,SEMANA,INSTITUCION,COD CANERO,NOMBRE CANERO,COD PROPIEDAD,NOMBRE PROPIEDAD,LOTE,HECTAREA,...,OPERACIÓN DETENIDA POR,COD DRONE,PILOTO 1,PILOTO 2,OBSERVACIONES,CONFORMIDAD CANERO,FINANCIADO,ID,OS,IDD
0,1,2024-07-04 14:50:55,27,69,18358,ROJAS AMELUNGE CARLOS,218,DON QUIJOTE,L4,16.738872,...,CONCLUIDO,DA 01,EDUARDO REYES,CHRISTIAN CALIZAYA,NaN,NaN,SIEMBRA 2024,1,25,30
1,1,2024-07-04 14:50:55,27,69,18358,ROJAS AMELUNGE CARLOS,218,DON QUIJOTE,L12.1,5.150160,...,CONCLUIDO,DA 01,EDUARDO REYES,CHRISTIAN CALIZAYA,NaN,NaN,SIEMBRA 2024,3,25,30
2,1,2024-07-04 14:50:55,27,69,18358,ROJAS AMELUNGE CARLOS,218,DON QUIJOTE,L9,32.409977,...,CONCLUIDO,DA 01,EDUARDO REYES,CHRISTIAN CALIZAYA,NaN,NaN,SIEMBRA 2024,4,25,30
3,1,2024-07-04 14:50:55,27,69,18358,ROJAS AMELUNGE CARLOS,218,DON QUIJOTE,L6,13.544786,...,CONCLUIDO,DA 01,EDUARDO REYES,CHRISTIAN CALIZAYA,NaN,NaN,SIEMBRA 2024,5,25,30
4,1,2024-07-04 14:50:55,27,69,18358,ROJAS AMELUNGE CARLOS,19,SANTA BARBARA,L16,5.915662,...,CONCLUIDO,DA 01,EDUARDO REYES,CHRISTIAN CALIZAYA,NaN,NaN,SIEMBRA 2024,2,25,30
5,1,2024-07-04 14:50:55,27,69,18358,ROJAS AMELUNGE CARLOS,19,SANTA BARBARA,L15,16.591264,...,CONCLUIDO,DA 01,EDUARDO REYES,CHRISTIAN CALIZAYA,NaN,NaN,SIEMBRA 2024,6,25,30


In [111]:
df_parte['FECHA'] = pd.to_datetime(df_parte['FECHA'], errors='coerce')
df_parte['HORA INICIO'] = pd.to_datetime(df_parte['HORA INICIO'], errors='coerce')
df_parte['HORA FINAL'] = pd.to_datetime(df_parte['HORA FINAL'], errors='coerce')

In [112]:
# Recorrer cada registro del DataFrame y actualizar los registros correspondientes en el archivo Excel
for idx, row in df_lib.iterrows():
    id_value = row['id']
    # Encontrar el índice correspondiente en el archivo Excel
    idx_parte = df_parte[df_parte['ID'] == id_value].index
    if not idx_parte.empty:
        idx_parte = idx_parte[0]
        # Convertir las fechas y eliminar la zona horaria
        fecha_registro = pd.to_datetime(row['fecha_registro'], errors='coerce').tz_localize(None)
        hora_inicio = pd.to_datetime(row['hora_inicio'], errors='coerce').tz_localize(None)
        hora_fin = pd.to_datetime(row['hora_fin'], errors='coerce').tz_localize(None)
        
        # Actualizar los valores en el archivo Excel
        df_parte.at[idx_parte, 'FECHA'] = fecha_registro
        df_parte.at[idx_parte, 'HORA INICIO'] = hora_inicio
        df_parte.at[idx_parte, 'HORA FINAL'] = hora_fin
        df_parte.at[idx_parte, 'TEMPERATURA'] = float(row['temperatura'])
        df_parte.at[idx_parte, 'VIENTO'] = float(row['viento'])
        df_parte.at[idx_parte, 'HUMEDAD'] = float(row['humedad'])
        df_parte.at[idx_parte, 'OPERACIÓN DETENIDA POR'] = str(row['estado_de_operacion'])
        df_parte.at[idx_parte, 'COD DRONE'] = str(row['dron'])
        df_parte.at[idx_parte, 'PILOTO 1'] = str(row['piloto_1'])
        df_parte.at[idx_parte, 'PILOTO 2'] = str(row['piloto_2'])
        df_parte.at[idx_parte, 'OBSERVACIONES'] = str(row['obs'])
        df_parte.at[idx_parte, 'IDD'] = float(row['idd'])
        df_parte.at[idx_parte, 'SEMANA'] = fecha_registro.isocalendar().week

C:\Users\bismarksr\AppData\Local\Temp\ipykernel_16724\2708270919.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'None' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_parte.at[idx_parte, 'OBSERVACIONES'] = str(row['obs'])


In [113]:
output_path = path_xlsx_parte
df_parte.to_excel(output_path, index=False)

# HACER COPIA Y ELIMINAR PARTE DIARIO Y PLANIFICACION DE AMIGOCLOUD

In [ ]:
id_proyecto = f'https://app.amigocloud.com/api/v1/projects/33457/sql'
for index, row in gdf_pla_gral.iterrows():
    wkb_hex = convertir_a_wkb(row['geometry'])
    insert_sql = f"""
    INSERT INTO dataset_345601 (id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, origen, dias, os, geometry)
    VALUES ({row['id']}, {row['unidad_01']}, '{row['unidad_02']}', {row['unidad_03']}, '{row['unidad_04']}', '{row['unidad_05']}', {row['area']}, '{row['origen']}', '{row['dias']}', '{row['os']}', ST_SetSRID(ST_GeomFromWKB('\\x{wkb_hex}'), 4326));
    """
    query_sql = {'query': insert_sql}
    resultado_post = amigocloud.post(id_proyecto, query_sql)

In [119]:
lista_idd

[30]

In [122]:
#copia y elimina los idd indicados del parte diario de amigocloud
for idd in lista_idd:
    id_proyecto = f'https://app.amigocloud.com/api/v1/projects/33457/sql'
    query = f'INSERT INTO dataset_348546 (fecha_registro, amigo_id, propiedad, piloto_1, piloto_2, lote, area, hora_inicio, hora_fin, institucion, labor, temperatura, viento, humedad, estado_de_operacion, dron, obs, canhero, idd, ubicaciones) SELECT fecha_registro, amigo_id, propiedad, piloto_1, piloto_2, lote, area, hora_inicio, hora_fin, institucion, labor, temperatura, viento, humedad, estado_de_operacion, dron, obs, canhero, idd, ubicaciones FROM dataset_342996 WHERE idd = {idd}'
    query_sql = {'query': query}
    post = amigocloud.post(id_proyecto, query_sql)
    query = {'query': f'DELETE FROM dataset_342996 WHERE idd = {idd}'}
    ejecutar = amigocloud.post(id_proyecto, query)
    print('se proceso IDD:', idd)

In [125]:
#copia y elimina los id indicados de la planificacion semanal de amigocloud
for id in lista_id:
    id_proyecto = f'https://app.amigocloud.com/api/v1/projects/33457/sql'
    query = f'INSERT INTO dataset_348547 (fecha_registro, amigo_id, id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, origen, dias, os, geometry) SELECT fecha_registro, amigo_id, id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, origen, dias, os, geometry FROM dataset_345601 WHERE id = {id}'
    query_sql = {'query': query}
    post = amigocloud.post(id_proyecto, query_sql)
    query = {'query': f'DELETE FROM dataset_345601 WHERE id = {id}'}
    ejecutar = amigocloud.post(id_proyecto, query)
    print('se proceso ID:', id)